In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
'''
Seasoning Trends: level='quarter'
Week Trends: level='weekday'
Time Trends: level='hour'
'''

raw_sql = textwrap.dedent("""
    SET DATEFIRST 1;
    SELECT DATEPART({level},timestamp1) AS {level},
           website,
           COUNT(url),
           COUNT(DISTINCT userid)
    FROM [data].[dbo].[health_records]
    GROUP BY DATEPART({level},timestamp1), website
    ORDER BY {level}, website
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
color_picker = {
    'haodf': 'rgb(55, 83, 109)',
    'cndzys': 'rgb(26, 118, 255)',
}

In [ ]:
title_switcher = {
    'quarter': 'Seasoning Trends',
    'weekday': 'Week Trends',
    'hour': 'Time Trends',
}

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_api')

In [ ]:
def to_df(level):
    sql = raw_sql.format(level=level)
    rows = cursor.execute(sql).fetchall()
    df = pd.DataFrame( [[ij for ij in i] for i in rows] )
    df.rename(columns={0: level, 1:'website', 2: 'Number of visits', 3:'Unique Visitors'}, inplace=True)
    return df

In [ ]:
def make_bars(df, website, target):
    is_website = df['website'] == website
    return go.Bar(
        x = df[is_website].iloc[:, 0],
        y = df[is_website][target],
        name = website,
        marker = dict(
            color = color_picker.get(website),
        )
    )

In [ ]:
def make_charts(level, target):
    df = to_df(level)
    haodf = make_bars(df, 'haodf', target)
    cndzys = make_bars(df, 'cndzys', target)
    data = [haodf, cndzys]
    layout = go.Layout(
        title=title_switcher.get(level),
        xaxis=dict(
            title=level,
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            ),
            nticks=40
        ),
        yaxis=dict(
            title=target,
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
        legend=dict(
            x=1.0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename=title_switcher.get(level)+' - '+target)

In [ ]:
make_charts('quarter', 'Number of visits')
make_charts('quarter', 'Unique Visitors')
make_charts('weekday', 'Number of visits')
make_charts('weekday', 'Unique Visitors')
make_charts('hour', 'Number of visits')
make_charts('hour', 'Unique Visitors')